# TensorFlow Learning (Python)

In [2]:
import numpy as np
import tensorflow as tf

print(tf.version.VERSION)
print(tf.version.COMPILER_VERSION)
print(tf.version.GRAPH_DEF_VERSION)

1.13.1
MSVC 190024215
27


# Tensor
A tensor is a generalization of vectors and matrices to potentially higher dimensions. Internally, TensorFlow represents tensors as n-dimensional arrays of base datatypes.

Each element in the Tensor has the same data type, and the data type is always known.

The rank of a tf.Tensor object is its number of dimensions.

Scalar - Rank 0 tensor

Vector - Rank 1 tensor

Matrix - Rank 2 tensot

#### Common Tensors
tf.Variable 

tf.constant

tf.placeholder

tf.SparseTensor 

#### dtype
QUANTIZED_DTYPES,  bfloat16,  bool,  complex128,  complex64,  double,  float16,  float32,  float64,  half,  int16,  int32,  int64,  int8,  qint16,  qint32,  qint8,  quint16,  quint8,  resource,  string,  uint16,  uint32,  uint64,  uint8,  variant

# Basics of TensorFlow

### zeros, ones and fill

In [7]:
zeros =  tf.zeros(3)
ones = tf.ones(6)
fill = tf.fill((2,2), value=9)

with tf.Session() as sess:
    print(sess.run(zeros),'\n')
    print(sess.run(ones),'\n')
    print(sess.run(fill),'\n')
    
    zeros =  tf.zeros((5,5))
    ones = tf.ones((2,4))
    fill = tf.fill((5,2,3), value=8.7)
    
    print(sess.run(zeros),'\n')
    print(sess.run(ones),'\n')
    print(sess.run(fill),'\n')

[0. 0. 0.] 

[1. 1. 1. 1. 1. 1.] 

[[9 9]
 [9 9]] 

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]] 

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]] 

[[[8.7 8.7 8.7]
  [8.7 8.7 8.7]]

 [[8.7 8.7 8.7]
  [8.7 8.7 8.7]]

 [[8.7 8.7 8.7]
  [8.7 8.7 8.7]]

 [[8.7 8.7 8.7]
  [8.7 8.7 8.7]]

 [[8.7 8.7 8.7]
  [8.7 8.7 8.7]]] 



### eye, range, diag

In [3]:
tf.InteractiveSession()   #Using the interactive session

In [6]:
eye = tf.eye(4)
eye.eval()

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [22]:
r = tf.range(5,8,1)    #tf.range(start, limit, step)    #limit excluded
r.eval()

array([5, 6, 7])

In [21]:
diag_elems = [1,1,2,2,3,3,4,4]
d = tf.diag(diag_elems)
d.eval()

array([[1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 2, 0, 0, 0, 0, 0],
       [0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 3, 0, 0, 0],
       [0, 0, 0, 0, 0, 3, 0, 0],
       [0, 0, 0, 0, 0, 0, 4, 0],
       [0, 0, 0, 0, 0, 0, 0, 4]])

In [23]:
d = tf.diag(r)
d.eval()

array([[5, 0, 0],
       [0, 6, 0],
       [0, 0, 7]])

In [7]:
rank_zero = tf.Variable(3, name='rank0_tensor')
rank_one = tf.Variable([1,2999,3], name='rank1_tensor')
rank_two = tf.Variable([[2,34,4],[5,67,7],[8,910,10]])
print(tf.rank(rank_zero))
print(tf.rank(rank_one))

element2 = rank_one[1]

print(element2)

with tf.Session() as sess:
    initVars = tf.global_variables_initializer()
    sess.run(initVars)
    print(sess.run(element2))
    print('Rank - ', sess.run(tf.rank(rank_zero)))
    print('Rank - ', sess.run(tf.rank(rank_one)))
    print('Rank - ', sess.run(tf.rank(rank_two)))

Instructions for updating:
Colocations handled automatically by placer.
Tensor("Rank:0", shape=(), dtype=int32)
Tensor("Rank_1:0", shape=(), dtype=int32)
Tensor("strided_slice:0", shape=(), dtype=int32)
2999
Rank -  0
Rank -  1
Rank -  2


# tf.random

In [16]:
normal_dist = tf.random.normal(shape=(1,10), mean=1, stddev=5, name='normal')
uniform_dist = tf.random.uniform(shape=(2,2), minval=9, maxval=15, dtype=tf.int32, seed=1, name='Uniform')

with tf.Session() as sess:
    print(sess.run(normal_dist))
    print(sess.run(uniform_dist))

[[-3.6625433  5.8312397  0.553386  -2.1982875  2.0972264  2.160311
  -7.352648   3.7380123  2.8522165  0.7527485]]
[[12 14]
 [12 13]]


# Basic Structure of Tensor Flow Program

You might think of TensorFlow Core programs as consisting of two discrete sections:

1. Building the computational graph (a tf.Graph).
2. Running the computational graph (using a tf.Session).

A computational graph is a series of TensorFlow operations arranged into a graph. The graph is composed of two types of objects.

1. tf.Operation (or "ops"): The nodes of the graph. Operations describe calculations that consume and produce tensors.
2. tf.Tensor: The edges in the graph. These represent the values that will flow through the graph. Most TensorFlow functions return tf.Tensors.

## Basic Structure (Code)

#### Construction phase (Create Tensors and Graphs)

In [8]:
a = tf.Variable(3, dtype=tf.float32, name='a')
b = tf.Variable(5, dtype=tf.float32, name='b')

# Y = abs(sin( (a^2 + b^2)^0.5  )   )
Y = tf.abs(tf.sin(tf.pow(tf.add(tf.multiply(a,a), tf.multiply(b,b)), 0.5))) #Create the computation graph
## Y = tf.abs(tf.sin(  (a**2 + b**2)**0.5  )) <----- You can also write it directly

In [9]:
initVars = tf.global_variables_initializer()

#### Execution Phase

In [10]:
sess = tf.Session() #Create a TF session
sess.run(initVars) #initlize variables
print(sess.run(Y)) #Execute the computation graph
sess.close() #Clsoe the session

0.4369757


# Math Operations

In [11]:
num = tf.constant(3, dtype=tf.float64)
aryX = tf.constant([3.1, 4.5, 5.7, 0, -1.2, -1.7, 0.1, 7.8])

sqrt_num = tf.sqrt(num, name="Square_root")
reciprocal_sqrt_num = tf.rsqrt(num, name="Reciprocal_square_root")
pow4_num = tf.pow(num, 4, name="Power")
log_num = tf.log(num, name="Log")
exp_num = tf.exp(num, name="Exp")
erf_num = tf.erf(num, name="error_function")


round_ary = tf.round(aryX)
rint_ary = tf.rint(aryX)
floor_ary = tf.floor(aryX)
ceil_ary = tf.ceil(aryX)


init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(sqrt_num))
    print(sess.run(reciprocal_sqrt_num))
    print(sess.run(pow4_num))
    print(sess.run(log_num))
    print(sess.run(exp_num))
    print(sess.run(erf_num))
    
    print(sess.run(round_ary))
    print(sess.run(rint_ary))
    print(sess.run(floor_ary))
    print(sess.run(ceil_ary))
    


1.7320508075688772
0.5773502691896258
81.0
1.0986122886681098
20.085536923187668
0.9999779095030014
[ 3.  4.  6.  0. -1. -2.  0.  8.]
[ 3.  4.  6.  0. -1. -2.  0.  8.]
[ 3.  4.  5.  0. -2. -2.  0.  7.]
[ 4.  5.  6.  0. -1. -1.  1.  8.]


## Matrix Operations

In [12]:
a = np.array([[1,2,3],[4,5,6],[7,8,9]])

a_a = tf.multiply(a,a)  # Elementwise multiplication
a_MatM_a = tf.matmul(a,a)
a_transpose = tf.transpose(a)


with tf.Session() as sess:
    print(sess.run(a_a))
    print(sess.run(a_MatM_a))
    print(sess.run(a_transpose))


[[ 1  4  9]
 [16 25 36]
 [49 64 81]]
[[ 30  36  42]
 [ 66  81  96]
 [102 126 150]]
[[1 4 7]
 [2 5 8]
 [3 6 9]]


## Gradients (autodiff)

In [13]:
a = tf.Variable(3, dtype=tf.float32, name='a')
b = tf.Variable(5, dtype=tf.float32, name='b')

Y = a**2 + b**3
g = tf.gradients(Y,[a,b])   #Calculates dY/da and dY/db (Partial Diff)

initVars = tf.global_variables_initializer()

sess = tf.Session()
sess.run(initVars)
sess.run(Y)
print(sess.run(g))
sess.close()

[6.0, 75.0]


## Gradient Descent Optimizer

In [14]:
x = tf.Variable(10, dtype=tf.float32, name='x')
three = tf.constant(3, dtype=tf.float32, name='three')
Y = (x - three)**2

#Create the GD optimizer object
GD_Optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.5)
training_op = GD_Optimizer.minimize(Y)

initVars = tf.global_variables_initializer()
sess = tf.Session()
sess.run(initVars)

for i in range(100): #Run 100 iterations of Gradient Descent
    sess.run(training_op) #One Step of gradient descent

print('x = ',sess.run(x))
sess.close()

x =  3.0


In [15]:
a = tf.Variable(10, dtype=tf.float32, name='x')
b = tf.Variable(180, dtype=tf.float32, name='y')
three = tf.constant(3, dtype=tf.float32, name='three')
Y = (a - three)**2 + (b - 50)**2

#Create the GD optimizer object
GD_Optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.5)
training_op = GD_Optimizer.minimize(Y)

initVars = tf.global_variables_initializer()
sess = tf.Session()
sess.run(initVars)

for i in range(100): #Run 100 iterations of Gradient Descent
    sess.run(training_op) #One Step of gradient descent

print('a = ',sess.run(a))
print('b = ',sess.run(b))
sess.close()

a =  3.0
b =  50.0
